In [1]:
import cv2
import numpy as np
# Load the pre-trained YOLOv4 model for vehicle detection
net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')

In [2]:
cap = cv2.VideoCapture('vehicle_movement.mp4')

In [3]:
output_path = 'output_new.mp4'

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get length of video in frames
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Create output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [4]:
length

316

In [5]:
roi = [(int(width * 0.45), int(height * 0.6)), (int(width * 0.55), int(height * 0.6)),
       (int(width * 0.8), int(height)), (int(width * 0.2), int(height))]

In [6]:
vehicle_counter = 0

In [8]:
frame_count = 0
while cap.isOpened() and frame_count < length:
    ret, frame = cap.read()
    if not ret:
        break


    # Preprocess frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255, (416, 416), swapRB=True, crop=False)

    # Set input to YOLOv4 model
    net.setInput(blob)

    # Forward pass through YOLOv4 model
    outputs = net.forward(net.getUnconnectedOutLayersNames())

    # Postprocess detections
    boxes = []
    confidences = []
    class_ids = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in indices:
        # j = i[0]
        x, y, w, h = boxes[i]
        label = class_ids[i]
        color = (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f"{label}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write processed frame to output video stream
    out.write(frame)

    print(f"Processed frame : {frame_count}")
    # Increase frame count
    frame_count += 1
    # Exit the video display window when the 'q' key is pressed
    cv2.imshow('Processed Frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the resources used for the video analysis
out.release()
cap.release()
cv2.destroyAllWindows()

In [8]:
frame_count

316